<a href="https://colab.research.google.com/github/RizanSM/zero_shot_llms_in_HIL_RL/blob/main/02_reacher_env/02_HF_DIRECT/03_Model_Testing_HF_DIRECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the required libraries in your Google Colab environment
!pip install gymnasium[mujoco] mujoco stable-baselines3 -q

In [ ]:
# Import the necessary libraries
import gymnasium as gym
import numpy as np
import pandas as pd
import pickle
import os
import matplotlib.pyplot as plt

In [ ]:
from stable_baselines3 import PPO
from google.colab import drive
from google.colab import data_table

# Mount Google Drive
drive.mount('/content/drive')

Average Episodic Reward (AER)

In [ ]:
def average_episodic_reward(test_trajectory_df):
    # Step 1: Calculate Cumulative Reward Per Episode
    cumulative_rewards = test_trajectory_df.groupby('Episode')['Reward'].sum().reset_index()
    cumulative_rewards.columns = ['episode', 'cumulative_reward']

    # Step 2: Calculate Variance and Standard Deviation of Cumulative Rewards
    variance = cumulative_rewards['cumulative_reward'].var()
    std_dev = cumulative_rewards['cumulative_reward'].std()
    final_reward = cumulative_rewards['cumulative_reward'].mean()

    # Step 4: Return the dataframe containing all results
    return cumulative_rewards, variance, std_dev, final_reward

COMBINING ALL THE DATA FRAMES

In [ ]:
def create_cummulative_reward(df1, df2, df3, df4, df5):
    """
    Create a data frame 'ideal_cummulative_reward' that contains:
      - 'episode' column (common across all data frames)
      - 'cumulative_reward_1' to 'cumulative_reward_5' columns from each data frame respectively,
        where each input data frame has columns 'episode' and 'cummulative_reward'.
      - 'mean_cumulative_reward' column containing the row-wise mean of the 5 cumulative rewards.

    Parameters:
      df1, df2, df3, df4, df5 (pd.DataFrame): Data frames with columns 'episode' and 'cummulative_reward'.

    Returns:
      pd.DataFrame: The merged and aggregated data frame.
    """
    # Rename the 'cummulative_reward' column in each data frame to a unique name.
    df1_renamed = df1.rename(columns={'cumulative_reward': 'cumulative_reward_1'})
    df2_renamed = df2.rename(columns={'cumulative_reward': 'cumulative_reward_2'})
    df3_renamed = df3.rename(columns={'cumulative_reward': 'cumulative_reward_3'})
    df4_renamed = df4.rename(columns={'cumulative_reward': 'cumulative_reward_4'})
    df5_renamed = df5.rename(columns={'cumulative_reward': 'cumulative_reward_5'})

    # Merge the data frames on the 'episode' column.
    merged_df = df1_renamed[['episode', 'cumulative_reward_1']].copy()
    merged_df = merged_df.merge(df2_renamed[['episode', 'cumulative_reward_2']], on='episode')
    merged_df = merged_df.merge(df3_renamed[['episode', 'cumulative_reward_3']], on='episode')
    merged_df = merged_df.merge(df4_renamed[['episode', 'cumulative_reward_4']], on='episode')
    merged_df = merged_df.merge(df5_renamed[['episode', 'cumulative_reward_5']], on='episode')

    # Compute the episode-wise mean of the cumulative rewards.
    reward_columns = [
        'cumulative_reward_1',
        'cumulative_reward_2',
        'cumulative_reward_3',
        'cumulative_reward_4',
        'cumulative_reward_5'
    ]
    merged_df['mean_cumulative_reward'] = merged_df[reward_columns].mean(axis=1)

    return merged_df

VARIANCE, STANDARD DEVIATION & FINAL AER (AVERAGE EPISODIC REWARD)

In [ ]:
def calculate_performance(variance_values, std_dev_values, final_reward_values):
    """
    Calculate performance metrics by computing the mean of each provided list.

    Parameters:
        variance_values (list of float): A list containing 5 variance values.
        std_dev_values (list of float): A list containing 5 standard deviation values.
        final_reward_values (list of float): A list containing 5 final reward values.

    Returns:
        tuple: A tuple containing three values:
            - mean_variance (float): Mean of the variance values.
            - mean_std_dev (float): Mean of the standard deviation values.
            - mean_final_reward (float): Mean of the final reward values.
    """
    mean_variance = sum(variance_values) / len(variance_values)
    mean_std_dev = sum(std_dev_values) / len(std_dev_values)
    mean_final_reward = sum(final_reward_values) / len(final_reward_values)


    return mean_variance, mean_std_dev, mean_final_reward


0. MODEL TESTING (HF_D_Ideal)

In [ ]:
trajectory_df_1 = pd.read_pickle('/content/drive/MyDrive/data3_rp1/3_test_trajectories/1_hf_d_ideal/1_hf_d_reacher_ideal_df_8.pkl')     # Update directory location 1
trajectory_df_2 = pd.read_pickle('/content/drive/MyDrive/data3_rp1/3_test_trajectories/1_hf_d_ideal/2_hf_d_reacher_ideal_df_8.pkl')     # Update directory location 2
trajectory_df_3 = pd.read_pickle('/content/drive/MyDrive/data3_rp1/3_test_trajectories/1_hf_d_ideal/3_hf_d_reacher_ideal_df_8.pkl')     # Update directory location 3
trajectory_df_4 = pd.read_pickle('/content/drive/MyDrive/data3_rp1/3_test_trajectories/1_hf_d_ideal/4_hf_d_reacher_ideal_df_8.pkl')     # Update directory location 4
trajectory_df_5 = pd.read_pickle('/content/drive/MyDrive/data3_rp1/3_test_trajectories/1_hf_d_ideal/5_hf_d_reacher_ideal_df_8.pkl')     # Update directory location 5

TESTING THE MODEL BY ANALYZING THE DATA FRAME <br>



1. Human feedback Direct Reacher Ideal data frame 1<br>
trajectory_df_1 ---> 1_hf_d_reacher_ideal_df

In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(trajectory_df_1)

A. Average Episodic Reward (trajectory_df_1)

In [ ]:
learning_curve_result_df_1, variance_1, std_dev_1, average_episodic_reward_1 = average_episodic_reward(trajectory_df_1)
print("Variance:", variance_1)
print("Standard Deviation:", std_dev_1)
print("Average Episodic Reward:", average_episodic_reward_1)

In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(learning_curve_result_df_1)

2. Human feedback Direct Reacher Ideal data frame 2<br>
trajectory_df_2 ---> 2_hf_d_reacher_ideal_df

In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(trajectory_df_2)

A. Average Episodic Reward (trajectory_df_2)

In [ ]:
learning_curve_result_df_2, variance_2, std_dev_2, average_episodic_reward_2 = average_episodic_reward(trajectory_df_2)
print("Variance:", variance_2)
print("Standard Deviation:", std_dev_2)
print("Average Episodic Reward:", average_episodic_reward_2)

In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(learning_curve_result_df_2)

3. Human feedback Direct Reacher Ideal data frame 3<br>
trajectory_df_3 ---> 3_hf_d_reacher_ideal_df

In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(trajectory_df_3)

A. Average Episodic Reward (trajectory_df_3)

In [ ]:
learning_curve_result_df_3, variance_3, std_dev_3, average_episodic_reward_3 = average_episodic_reward(trajectory_df_3)
print("Variance:", variance_3)
print("Standard Deviation:", std_dev_3)
print("Average Episodic Reward:", average_episodic_reward_3)

In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(learning_curve_result_df_3)

4. Human feedback Direct Reacher Ideal data frame 4<br>
trajectory_df_4 ---> 4_hf_d_reacher_ideal_df

In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(trajectory_df_4)

A. Average Episodic Reward (trajectory_df_4)

In [ ]:
learning_curve_result_df_4, variance_4, std_dev_4, average_episodic_reward_4 = average_episodic_reward(trajectory_df_4)
print("Variance:", variance_4)
print("Standard Deviation:", std_dev_4)
print("Average Episodic Reward:", average_episodic_reward_4)

In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(learning_curve_result_df_4)

5. Human feedback Direct Reacher Ideal data frame 5<br>
trajectory_df_5 ---> 5_hf_d_reacher_ideal_df

In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(trajectory_df_5)

A. Average Episodic Reward (trajectory_df_5)

In [ ]:
learning_curve_result_df_5, variance_5, std_dev_5, average_episodic_reward_5 = average_episodic_reward(trajectory_df_5)
print("Variance:", variance_5)
print("Standard Deviation:", std_dev_5)
print("Average Episodic Reward:", average_episodic_reward_5)

In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(learning_curve_result_df_5)

COMBINED DATA FRAMES - HF-D-IDEAL

In [ ]:
cummulative_reward = create_cummulative_reward(
        learning_curve_result_df_1,
        learning_curve_result_df_2,
        learning_curve_result_df_3,
        learning_curve_result_df_4,
        learning_curve_result_df_5
    )

data_table.enable_dataframe_formatter()
data_table.DataTable(cummulative_reward)

VARIANCE, STANDARD DEVIATION & FINAL REWARD : HF-D-IDEAL

In [ ]:
 # Organize the values into lists
variance_list = [variance_1, variance_2, variance_3, variance_4, variance_5]
std_dev_list = [std_dev_1, std_dev_2, std_dev_3, std_dev_4, std_dev_5]
final_reward_list = [average_episodic_reward_1, average_episodic_reward_2, average_episodic_reward_3, average_episodic_reward_4, average_episodic_reward_5]
# Call the function to compute the mean performance values
mean_variance, mean_std_dev, mean_final_reward = calculate_performance(variance_list, std_dev_list, final_reward_list)

# Print the results
print("HF_D_IDEAL Mean Variance:", mean_variance)
print("HF_D_IDEAL  Mean Standard Deviation:", mean_std_dev)
print("HF_D_IDEAL  Mean Average Episodic Reward:", mean_final_reward)
